# Fortgeschrittene Git-Funktionen: Git Large File Storage (LFS)

## Einführung

In der digitalen Geschichtswissenschaft arbeiten wir zunehmend mit großen Dateien, insbesondere wenn wir:
- Embedding-Dateien für RAG-Systeme generieren
- Große Textkorpora verarbeiten
- Mit Bild- oder Audiodateien arbeiten
- Trainierte Modelle speichern

Git, unser Standard-Werkzeug für Versionskontrolle, wurde ursprünglich für Quellcode entwickelt - also für kleine Textdateien. Bei großen Dateien (>100MB) stößt Git jedoch an seine Grenzen. Hier kommt Git Large File Storage (Git LFS) ins Spiel.

## Was ist Git LFS?

Git LFS ist eine Erweiterung für Git, die speziell für die Verwaltung großer Dateien entwickelt wurde. Statt die großen Dateien direkt im Git-Repository zu speichern, werden sie in einem separaten Speicher abgelegt und durch kleine Pointer-Dateien im Repository referenziert. Wichtig zu verstehen: Die großen Dateien werden dabei weiterhin versioniert - jede Version einer LFS-Datei wird gespeichert und kann wiederhergestellt werden, genau wie bei normalen Git-Dateien. Der Unterschied liegt nur in der Art der Speicherung und Übertragung.

<div class="mermaid">
graph TD
    A[Git Repository] -->|Enthält| B[Quellcode]
    A -->|Enthält| C[Pointer-Datei]
    C -->|Verweist auf| D[LFS Speicher]
    D -->|Enthält| E[Große Datei]
    style D fill:#4974a5,stroke:#333,stroke-width:4px
    E -->|Version 1| F[V1 der großen Datei]
    E -->|Version 2| G[V2 der großen Datei]
    E -->|Version 3| H[V3 der großen Datei]
</div>

## Warum brauchen wir Git LFS?

Konkrete Beispiele aus der Digital Humanities Praxis:

1. **RAG-Systeme & Embeddings**
   - Eine typische Embedding-Datei für einen Textkorpus kann leicht 500MB-1GB groß werden
   - Ohne LFS würde dies:
     - Das Repository aufblähen
     - Klonen und Pushen verlangsamen
     - Möglicherweise die Größenlimits von GitHub überschreiten

2. **Kollaborative Forschung mit großen Dateien**
   - Effiziente Zusammenarbeit an großen Datensätzen ohne das komplette Repository herunterladen zu müssen
   - Selektives Herunterladen nur der benötigten großen Dateien
   - Bandbreitenoptimierung bei der Arbeit mit Remote-Repositories
   - Vermeidung von Repository-Bloat bei häufigen Updates großer Dateien

## Versionierung in Git LFS

Ein häufiges Missverständnis ist, dass LFS-Dateien nicht versioniert würden. Das stimmt nicht:

1. **Vollständige Versionierung**
   - Jede Änderung an einer LFS-Datei wird als neue Version gespeichert
   - Sie können jederzeit zu älteren Versionen zurückkehren
   - Die komplette Historie bleibt erhalten

2. **Unterschied zur normalen Git-Versionierung**
   - Bei normalen Git-Dateien: Alle Versionen im Hauptrepository
   - Bei LFS-Dateien: Versionen im LFS-Speicher, nur Pointer im Hauptrepository
   - Das Ergebnis ist das gleiche: Sie haben Zugriff auf alle Versionen

3. **Praktische Bedeutung**
   - Sie können `git log` für LFS-Dateien nutzen
   - `git checkout` funktioniert wie gewohnt
   - Branches können verschiedene Versionen der LFS-Dateien enthalten

## Wo finde ich weitere Informationen?

- [Offizielle Git LFS Dokumentation](https://git-lfs.com/)
- [GitHub's Guide zu Git LFS](https://docs.github.com/en/repositories/working-with-files/managing-large-files/about-git-large-file-storage)
- [GitLab LFS Dokumentation](https://docs.gitlab.com/ee/topics/git/lfs/)

## Voraussetzungen

Bevor wir mit den praktischen Übungen beginnen, stellen Sie sicher, dass Sie:
- Git auf Ihrem System installiert haben
- Grundlegende Git-Befehle kennen (clone, add, commit, push, pull)
- Zugriff auf ein Git-Repository haben (z.B. auf GitHub oder GitLab)

In den folgenden Abschnitten werden wir:
1. Git LFS installieren und konfigurieren
2. Lernen, wie man große Dateien tracked
3. Best Practices für die Arbeit mit LFS kennenlernen
4. Einen typischen Workflow mit Embedding-Dateien durchgehen

# Installation und Einrichtung von Git LFS

## Installation

Die Installation von Git LFS unterscheidet sich je nach Betriebssystem. Wählen Sie die für Sie passende Methode:

### Windows

1. **Automatische Installation mit Git for Windows**
   - Wenn Sie Git neu installieren: Git LFS ist bereits in den neueren Versionen von [Git for Windows](https://gitforwindows.org/) enthalten
   - Bei der Installation "Git LFS" Option aktivieren

2. **Manuelle Installation**
   - Laden Sie den Installer von der [Git LFS Releases](https://github.com/git-lfs/git-lfs/releases/latest) Seite herunter
   - Führen Sie die .exe Datei aus

### macOS

Mit Homebrew (empfohlen):
```bash
brew install git-lfs
```

### Linux (Ubuntu/Debian)
```bash
sudo apt-get update
sudo apt-get install git-lfs
```

### Überprüfung der Installation

Nach der Installation können Sie in einem Terminal/der Kommandozeile überprüfen, ob Git LFS korrekt installiert wurde:

```bash
git lfs version
```

Die Ausgabe sollte etwa so aussehen:
```
git-lfs/3.4.0 (GitHub; linux amd64; go 1.20.3)
```

## Einrichtung

### 1. Globale Einrichtung

Nach der Installation müssen Sie Git LFS einmalig global einrichten:

```bash
git lfs install
```

Erfolgreiche Ausgabe:
```
Updated Git hooks.
Git LFS initialized.
```

### 2. Repository-spezifische Einrichtung

Für jedes Repository, in dem Sie Git LFS nutzen möchten:

1. Navigieren Sie zum Repository:
```bash
cd mein-repository
```

2. Initialisieren Sie Git LFS für das Repository:
```bash
git lfs install
```

3. Legen Sie fest, welche Dateitypen mit LFS verwaltet werden sollen:
```bash
# Beispiel für Embedding-Dateien
git lfs track "*.npy"
git lfs track "*.bin"
git lfs track "*.h5"
```

4. Eine `.gitattributes` wird automatisch erstellt und muss zum Repository hinzugefügt werden:
```bash
git add .gitattributes
git commit -m "Configure Git LFS tracking"
```

## Überprüfung der Einrichtung

Sie können überprüfen, welche Dateitypen für LFS konfiguriert sind:
```bash
git lfs track
```

Ausgabe sollte die konfigurierten Patterns zeigen:
```
Listing tracked patterns
    *.npy (.gitattributes)
    *.bin (.gitattributes)
    *.h5 (.gitattributes)
```


## Wichtige Hinweise

1. **Speicherplatzbedarf**
   - Git LFS benötigt lokalen Speicherplatz für den LFS Cache
   - Standard: `~/.git-lfs` (Linux/macOS) oder `%LocalAppData%\GitLfs` (Windows)
   - Cache-Größe kann konfiguriert werden

2. **HU Berlin GitLab Limits**
   - Gesamtspeicher pro Namespace: 10.0 GiB (inkl. Repository und LFS)
   - Maximale Push-Größe: 5 GiB
   - Maximale Import-Größe: 5 GiB
   - Maximale Dateigröße für Anhänge: 100 MiB

   Diese Limits gelten für:
   - Repository-Inhalte
   - LFS-Objekte
   - Container Registry
   - Packages
   - Wikis

   *Tipp: Sie können Ihre aktuelle Speichernutzung unter "[Settings > Usage Quotas](https://scm.cms.hu-berlin.de/-/profile/usage_quotas#storage-quota-tab)" in GitLab überprüfen.*

3. **Andere Hosting-Plattformen**
   - [GitHub (kostenlos)](https://docs.github.com/en/repositories/working-with-files/managing-large-files/about-storage-and-bandwidth-usage)

3. **Bandbreite**
   - Große LFS-Dateien benötigen entsprechende Bandbreite
   - Selective Checkout möglich (siehe nächstes Kapitel)

## Troubleshooting

### Häufige Probleme und Lösungen

1. **"git-lfs not found"**
   ```bash
   # Prüfen Sie den PATH
   echo $PATH
   # Oder installieren Sie neu
   git lfs install
   ```

2. **Konflikt mit existierenden Dateien**
   ```bash
   # Bereits gepushte große Dateien erst entfernen
   git rm --cached großeDatei.npy
   # Dann neu hinzufügen
   git add großeDatei.npy
   ```

### Support und Hilfe

- [Offizielle Git LFS Issues](https://github.com/git-lfs/git-lfs/issues)
- [Git LFS FAQ](https://github.com/git-lfs/git-lfs/blob/main/docs/man/git-lfs-faq.adoc)
- [GitLab Dokumentation](https://docs.gitlab.com/ee/topics/git/lfs/)

## Speicherplatz-Management im HU GitLab

### Best Practices für große Dateien

1. **Vor dem Push prüfen**
   ```bash
   # Größe der LFS-Objekte anzeigen
   git lfs ls-files --size
   ```

2. **Regelmäßige Überprüfung**
   - Überprüfen Sie regelmäßig Ihre Speichernutzung in GitLab
   - Navigieren Sie zu: Settings > Usage Quotas
   - Achten Sie besonders auf:
     - Repository-Größe
     - LFS-Objekte
     - Gesamtnutzung

3. **Strategien zur Speicheroptimierung**
   - Große Datensätze in kleinere Teile aufteilen
   - Nicht mehr benötigte LFS-Objekte bereinigen
   - Temporäre oder generierte Dateien im `.gitignore` ausschließen
   - Embedding-Dateien komprimieren, wenn möglich

4. **Alternative Speicherlösungen**
   - Für sehr große Datensätze: HU Box oder andere Speicherdienste nutzen
   - Links zu externen Speicherorten im Repository dokumentieren

## Nächste Schritte

Nach erfolgreicher Installation und Einrichtung können Sie:
1. Große Dateien zu Ihrem Repository hinzufügen
2. Mit LFS-Dateien arbeiten wie mit normalen Git-Dateien
3. Erweiterte Funktionen nutzen (siehe nächstes Kapitel)

Lassen Sie uns im nächsten Abschnitt einen typischen Workflow mit Git LFS durchgehen.

# Grundlegende Konzepte von Git LFS

## Wie Git LFS funktioniert

Git LFS verwendet ein System von Pointer-Dateien und Remote Storage. Lassen Sie uns Schritt für Schritt durchgehen, wie das funktioniert:

### Die Pointer-Datei

<div class="mermaid">
graph LR
    A[Große Datei] -->|wird ersetzt durch| B[Pointer-Datei]
    B -->|verweist auf| C[LFS Storage]
    C -->|enthält| A
</div>

Eine Pointer-Datei ist eine kleine Textdatei, die anstelle der großen Datei im Git Repository gespeichert wird. Sie sieht etwa so aus:

```text
version https://git-lfs.github.com/spec/v1
oid sha256:4b9458b125a3b60f387e92f0f212d9548000262385434785477be6ecd1c7edab
size 1234567
```

Die Pointer-Datei enthält:
- Die LFS-Version
- Einen eindeutigen Hash (OID) der eigentlichen Datei
- Die Größe der eigentlichen Datei

### Der Workflow

Wenn Sie mit Git LFS arbeiten, passiert Folgendes:

1. **Beim Hinzufügen einer Datei** (git add)
<div class="mermaid">
sequenceDiagram
    participant U as User
    participant G as Git
    participant L as LFS
    U->>G: git add großeDatei.npy
    G->>L: Prüfe LFS-Tracking
    L->>L: Erstelle Hash
    L->>L: Speichere Datei im LFS Cache
    L->>G: Erstelle Pointer-Datei
</div>

2. **Beim Push** (git push)
<div class="mermaid">
sequenceDiagram
    participant L as Lokales Repo
    participant G as Git Server
    participant S as LFS Storage
    L->>G: Push Pointer-Datei
    L->>S: Upload große Datei
    S-->>G: Bestätige Storage
</div>

3. **Beim Pull** (git pull)
<div class="mermaid">
sequenceDiagram
    participant U as User
    participant G as Git
    participant L as LFS
    participant S as LFS Storage
    U->>G: git pull
    G->>G: Update Pointer-Dateien
    G->>L: Prüfe LFS-Dateien
    L->>S: Download wenn nötig
</div>

## Praktische Beispiele

### 1. Embedding-Datei tracken

Nehmen wir an, Sie haben eine Embedding-Datei für Ihren Textkorpus:

```bash
# Datei zum LFS-Tracking hinzufügen
git lfs track "embeddings.npy"

# Status überprüfen
git lfs status

# Datei hinzufügen wie gewohnt
git add embeddings.npy
git commit -m "Add embeddings for text corpus"
```

### 2. Pointer-Datei inspizieren

Sie können sich den Inhalt einer Pointer-Datei ansehen:

```bash
# Zeigt den Inhalt der Pointer-Datei
git lfs pointer --file embeddings.npy
```

### 3. LFS-Dateien auflisten

```bash
# Alle LFS-Dateien anzeigen
git lfs ls-files

# Mit Details
git lfs ls-files --size --format "%(size) %(name)"
```

## Besonderheiten und Hinweise

### Smudge und Clean Filter

Git LFS nutzt zwei wichtige Filter:

1. **Clean Filter**: Konvertiert große Dateien in Pointer beim Staging
2. **Smudge Filter**: Ersetzt Pointer durch echte Dateien beim Checkout

<div class="mermaid">
graph TD
    A[Arbeitsverzeichnis] -->|Clean| B[Git Repository]
    B -->|Smudge| A
    B -->|Pointer| C[LFS Storage]
    C -->|Große Datei| A
</div>

### Partial Clone und Fetch

Git LFS ermöglicht auch:
- Selektives Herunterladen von LFS-Dateien
- Verzögertes Laden großer Dateien
- Priorisierung bestimmter LFS-Dateien

```bash
# Nur bestimmte LFS-Dateien holen
git lfs fetch --include="path/to/file"

# Alle LFS-Dateien eines Commits holen
git lfs pull
```

## Häufige Fragen

1. **Wie groß sollte eine Datei sein, um LFS zu nutzen?**
   - Faustregel: > 100MB
   - Abhängig von Ihrem Workflow
   - Beachten Sie Repository-Limits

2. **Dateitypen und Kompression**

   a) **Allgemeines Verhalten**
   - LFS versucht Dateien beim Transfer zu komprimieren
   - Im Storage werden vorher unkrompierte Dateien weiter unkomprimiert gespeichert
   
   b) **Bereits komprimierte Formate**
   - Dateien wie `.npz`, `.zip`, `.gz`, `.jpg`, `.png` sind bereits komprimiert
   - LFS erkennt dies und überspringt die Transferkompression
   - Keine zusätzliche Speicherersparnis durch LFS möglich
   

3. **Was passiert bei Konflikten?**
   - LFS-Dateien werden wie normale Dateien behandelt
   - Git's übliche Konfliktlösung greift
   - Binary-Konflikte erfordern manuelle Auflösung

# Praktische Anwendung von Git LFS

## Typische Workflows in den Digital Humanities

### 1. Einrichten eines neuen Projekts

```bash
# Neues Repository erstellen
git init mein-dh-projekt
cd mein-dh-projekt

# Git LFS aktivieren
git lfs install

# Typische Dateitypen für DH-Projekte tracken
git lfs track "*.pkl"  # Für Embeddings und Modelle
git lfs track "*.npz"  # Für komprimierte NumPy Arrays
git lfs track "*.csv"  # Für große Datensätze
git lfs track "*.xml"  # Für große XML-Korpora

# .gitattributes committen
git add .gitattributes
git commit -m "Configure Git LFS tracking"
```

### 2. Arbeiten mit Embedding-Dateien

Ein typischer Workflow für RAG-Systeme:

```python
import numpy as np
from sentence_transformers import SentenceTransformer

# Embeddings erstellen
model = SentenceTransformer('all-MiniLM-L6-v2')
texts = ["Ihre", "Beispiel", "Texte"]
embeddings = model.encode(texts)

# Als NPZ speichern (bereits komprimiert)
np.savez_compressed('embeddings.npz', embeddings=embeddings)
```

```bash
# Embedding-Datei zu Git hinzufügen
git add embeddings.npz
git commit -m "Add text corpus embeddings"
git push origin main
```

### 3. Kollaboratives Arbeiten

Wenn mehrere Personen am Projekt arbeiten:

```bash
# Repository klonen
git clone https://gitlab.hu-berlin.de/projekt/name.git

# LFS-Dateien initial nicht herunterladen
git lfs install --skip-smudge

# Nur bestimmte LFS-Dateien holen
git lfs pull --include="embeddings/wichtig.npz"

# Später alle LFS-Dateien nachladen
git lfs pull
```

### 4. Größere Datensätze verwalten

Für Projekte mit mehreren großen Dateien:

```bash
# Überblick über LFS-Dateien bekommen
git lfs ls-files --size

# Speichernutzung analysieren
du -sh .git/lfs/objects/

# Bestimmte Dateitypen vom LFS-Tracking ausschließen
echo "temp/*.npz" >> .gitignore
```

## Best Practices

### 1. Repository-Struktur

Mögliche Struktur für DH-Projekte mit LFS:

```
projekt/
├── data/
│   ├── raw/                # Rohdaten (nicht in LFS)
│   ├── processed/          # Verarbeitete Daten
│   └── embeddings/         # LFS-getrackte Embeddings
├── notebooks/              # Jupyter Notebooks
├── .gitattributes          # LFS-Konfiguration
└── .gitignore              # Ignorierte Dateien
```

### 2. LFS-Dateien dokumentieren

Erstellen Sie eine `lfs-files.md`:

```markdown
# LFS-Dateien in diesem Repository

## embeddings/corpus_embeddings.npz
- Erstellt mit: all-MiniLM-L6-v2
- Datum: 2024-02-10
- Größe: 2.3GB
- Verwendung: RAG-System für historische Dokumente

## models/finetuned_model.pkl
- Basis: BERT-base-german-cased
- Trainingskorpus: Deutsche Zeitungen 1850-1900
- Größe: 1.1GB
```

### 3. Versionierung von Datensätzen

```bash
# Neue Version eines Datensatzes hinzufügen
git add data/processed/dataset_v2.npz
git commit -m "Update dataset with additional sources

- Added newspapers from 1860-1865
- Improved OCR quality
- Size: 3.2GB"

# Zwischen Versionen wechseln
git checkout v1.0 data/processed/dataset.npz
```

### 4. Performance-Optimierung

```bash
# Lokalen LFS-Cache aufräumen
git lfs prune

# Nur benötigte LFS-Dateien herunterladen
git clone --filter=blob:none --sparse <url>
git sparse-checkout set data/embeddings/wichtig.npz
```

## Fehlerbehebung

### Häufige Probleme und Lösungen

1. **Push schlägt fehl wegen Größenlimit**
```bash
# Prüfen der Dateigrößen
git lfs ls-files --size

# Große Dateien aufteilen
split -b 2G große_datei.npz teil_
```

2. **Langsamer Download**
```bash
# Nur notwendige Dateien laden
git lfs fetch --include="pfad/zur/datei"
```

3. **Konflikte in LFS-Dateien**
```bash
# Aktuelle Version behalten
git checkout --ours pfad/zur/datei.npz
# ODER Remote-Version übernehmen
git checkout --theirs pfad/zur/datei.npz
```

## Nützliche Skripte

### LFS-Status überprüfen

```python
import subprocess
import json

def get_lfs_status():
    """Überprüft Status der LFS-Dateien im Repository."""
    result = subprocess.run(['git', 'lfs', 'ls-files', '--json'],
                          capture_output=True, text=True)
    files = json.loads(result.stdout)
    
    total_size = 0
    print("LFS-Dateien im Repository:")
    print("-" * 50)
    
    for file in files:
        size_mb = file['size'] / (1024 * 1024)
        total_size += size_mb
        print(f"{file['path']}: {size_mb:.2f} MB")
    
    print("-" * 50)
    print(f"Gesamtgröße: {total_size:.2f} MB")

if __name__ == "__main__":
    get_lfs_status()
```

# Fazit

Git LFS ist ein mächtiges Werkzeug für die Digital Humanities, das uns erlaubt, große Dateien effizient zu verwalten. Mit den gelernten Konzepten und Praktiken können Sie:
- Große Datensets verwalten
- Kollaborativ arbeiten
- Forschung reproduzierbar machen
- Effizient mit Ressourcen umgehen